In [1]:
import os
import dotenv
import warnings

from langchain_groq import ChatGroq
from langchain.schema import Document
from langchain_qdrant import QdrantVectorStore
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [2]:
warnings.simplefilter("ignore")

# Load environment variables
dotenv.load_dotenv() # To load the Groq API from .env
os.environ["TOKENIZERS_PARALLELISM"] = "false" # To suppress HuggingFace warnings

# Constants
EMBED_MODEL = "sentence-transformers/all-mpnet-base-v2"

QDRANT_PORT = 6333
QDRANT_HOST = "localhost"
QDRANT_COLLECTION = "naive-rag"

In [3]:
# Load the document
doc_path = "./data/document.txt"
with open(doc_path, mode='r') as f:
    text = f.read()

# Create the Document and metadata
metadata = {
    "source": "https://www.ibm.com/br-pt/think/topics/geographic-information-system"
}

documents = [Document(page_content=text, metadata=metadata)]

In [4]:
# Visualizing
documents[0].metadata.get("source")

'https://www.ibm.com/br-pt/think/topics/geographic-information-system'

In [5]:
# Perform the splitter step
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,    # Number of character per chunk
    chunk_overlap=100, # Number of character overlapping the next chunk
    length_function=len
)

chunks = text_splitter.split_documents(documents)

In [6]:
# Visualizing
chunks[0].page_content

'O que é SIG (sistema de informação geográfica)?\nSistemas de informação geográfica (GIS) são sistemas computacionais que produzem visualizações conectadas de dados geoespaciais, ou seja, dados referenciados espacialmente à Terra. Além de criar visualizações, o GIS também permite capturar, armazenar, analisar e gerenciar esses dados.'

In [7]:
# Create the Embedding and the QDrant clietn
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
qdrant = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT) # Via docker

# Create a collection
qdrant.create_collection(
    collection_name=QDRANT_COLLECTION,
    vectors_config=VectorParams(size=768, distance=Distance.COSINE) # Cosine Similarity
)

True

In [8]:
# Store the data in the Vector Store from LangChain
vector_store = QdrantVectorStore(
    client=qdrant,
    embedding=embeddings,
    collection_name=QDRANT_COLLECTION
)

vector_store.add_documents(chunks)

['d99a119ab70445e683dbaabf6ae96255',
 '31a410b7f948462999b227226422d63d',
 'c7c692bd9dce4b61bc52c7740dec39ce',
 'c0687e47a2344202ac32e5b7d7750a71',
 '35bca2912b1d4aaca4298130d0c14fe2',
 '8cce53256ebf4ebcaea2dd6aa924d496',
 '48619484b66d4ed48e8b8267c6a51b87',
 '5d33ae8781da45c0a5db04ae459b1347',
 '22415ee3fb9b499a852fb11a0f483faa',
 'a6985c0e73da4341a80066f0b4c58b2d',
 '2bcadd764f8e419bb581886bacdec532',
 'acc4f564eaa04a39a7eaf6ef3ed27700',
 '70b6d30db0b44c658de3e1ee45bb7a23',
 'cf261c97c0454fca979e5b5a85156b58',
 '22ef2d6ff85144a694edb7cb27de4948',
 '2ddb50c5144041039f8e33ae0a51207c',
 '0ecf2208f3f3478587d94d1453a0244d',
 '9d779efacbbb4977bc9f5aaddb560ee6',
 'd1cb8d931cf4471c87091f3edc100837',
 '6c37f7eb98de4f60a3da131bbd603a73',
 '33c23558f2e3430c9639b940dc95c967',
 '3d1ec65aedd84d67a9a8dc61ceed7183',
 '2bca76924b884ec4b3ebe5a8ca6fa29d',
 'be56344cc8aa44bbbd7ba297ae4b9f60',
 '765b9fa17d9c48c7a38cf54e8f27eca0',
 '8558063c829b41069fda7ceab9d9b2de',
 '9a52a7e91c8343aba3d20a593aa5ccaa',
 

In [9]:
# Fast example test
query = "O que é SIG?"
vector_store.similarity_search_with_relevance_scores(query=query, k=1)[0]

(Document(metadata={'source': 'https://www.ibm.com/br-pt/think/topics/geographic-information-system', '_id': 'd99a119a-b704-45e6-83db-aabf6ae96255', '_collection_name': 'naive-rag'}, page_content='O que é SIG (sistema de informação geográfica)?\nSistemas de informação geográfica (GIS) são sistemas computacionais que produzem visualizações conectadas de dados geoespaciais, ou seja, dados referenciados espacialmente à Terra. Além de criar visualizações, o GIS também permite capturar, armazenar, analisar e gerenciar esses dados.'),
 0.7807032)

In [10]:
# Simple prompt template for RAG based
template = PromptTemplate.from_template("""
Answer the query based on the context. If you don't know, just say you don't know.
All answers must be in pt-BR.

### CONTEXT:
{context}

### QUERY:
{query}                 
""")

In [11]:
# Load the LLM from Groq
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.1
)

In [12]:
# Query function
def naive_rag_query(query: str) -> dict:
    """
    Perform a query using the retriaval data as context.

    args:
        query: str: User query

    returns:
        A dict with output text and your relevance score.
    """
    context = vector_store.similarity_search_with_relevance_scores(query=query, k=5)
    # Filter to get only sentences with relevance score >= 0.7
    context = list(filter(lambda x: x[1] >= 0.7, context)) # x[0]->Text, x[1]->Score

    # i.e the query doesn't make sense in the context
    if len(context) == 0:
        return {
            "text": "\nEu não sou capaz de responder a isso.\n",
            "score": 0.0
        }
    
    chain = template | llm | StrOutputParser()
    response = chain.invoke({"context": context, "query": query})

    return {
        "text": "\n" + response + "\n",
        "score": context[0][1]
    }

In [13]:
# Unsatisfied
response_1 = naive_rag_query(query="Que horas são?")

print(f"Text: {response_1.get('text')}")
print(f"Score: {response_1.get('score')}")

Text: 
Não sei.

Score: 0.7249676899999999


In [14]:
# Satisfied
response_2 = naive_rag_query(query="O que é SIG?")

print(f"Text: {response_2.get('text')}")
print(f"Score: {response_2.get('score')}")

Text: 
Um sistema de informação geográfica (SIG) é um sistema computacional que produz visualizações conectadas de dados geoespaciais, ou seja, dados referenciados espacialmente à Terra. Além de criar visualizações, o SIG também permite capturar, armazenar, analisar e gerenciar esses dados.

Score: 0.7807032


In [15]:
# Satisfied
response_3 = naive_rag_query(query="Me fale sobre a sobreposição.")

print(f"Text: {response_3.get('text')}")
print(f"Score: {response_3.get('score')}")

Text: 
A sobreposição não é mencionada explicitamente no contexto fornecido. Portanto, não tenho conhecimento sobre a sobreposição no contexto de um sistema de informação geográfica (GIS).

Score: 0.7267544


In [16]:
# Satisfied
response_4 = naive_rag_query(query="Me fale sobre sensoriamento remoto.")

print(f"Text: {response_4.get('text')}")
print(f"Score: {response_4.get('score')}")

Text: 
O sensoriamento remoto é um processo que coleta dados geoespaciais e realiza medições da superfície terrestre a partir do alto, utilizando sensores em satélites, balões, drones e aviões para captar e registrar a energia refletida ou emitida. Existem dois tipos de detecção remota: ativa e passiva. O sensoriamento remoto ativo utiliza sensores que emitem sua própria fonte de energia ou luz e detectam a radiação refletida, enquanto o sensoriamento remoto passivo não emite energia própria e coleta a radiação natural emitida e refletida.

Score: 0.8150071999999999
